In [8]:
#importando bibliotecas
import numpy as np
import pandas as pd
from operator import add
import pyspark
import os.path
import re

# carregar base de dados
fileName = os.path.join('data', 'CommentsApril2018.csv')
df = pd.read_csv(fileName)
df_sample = (df.sample(frac=0.01, replace=True))

/home/Drii/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()

def data_prep(df):
    
    new_df = df[['articleID','commentBody', 'editorsSelection']]
    new_df['by_words'] = new_df['commentBody'].apply(lambda x: trat_words(removerPontuacao(x).split()))
    del new_df['commentBody']
    #new_df = new_df.apply(lambda x: (x[0], x[1], trat_words(x[2])))
    
    return new_df.values

In [14]:
RDD_sample = data_prep(df_sample)

/home/Drii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
RDD_top_words = sc.parallelize(map(lambda x : tuple (x),RDD_sample))

[('5ac90e33068401528a2a494a', False, ['have', 'in', 'and', 'the', 'to', 'colleges', 'four', 'year', 'less', 'one'])]


In [30]:
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint


RDD_train, RDD_test = RDD_top_words.randomSplit([0.6, 0.4])

model = NaiveBayes.train(RDD_train, 1.0)

predictionAndLabel = RDD_test.map(lambda p: (model.predict(p.features), p.label))

accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / RDD_test.count()

print('Acurácia do Modelo {}'.format(accuracy))


ValueError: `data` should be an RDD of LabeledPoint

In [61]:
from pyspark.mllib.regression import LabeledPoint
parsedData = list(map(lambda p: LabeledPoint(p[1], p[2]), RDD_sample))

ValueError: could not convert string to float: 'have'

In [1]:
def OHE_auto(raw):
    return raw_derived = ''
    

SyntaxError: invalid syntax (<ipython-input-1-e90d9533d25f>, line 2)